In [1]:
# set working directory to dashboard
import os
%cd ..
print(f'Using {os.getcwd()} as the working directory.')

/Users/anthonyyoskovich/bezosbestfriends
Using /Users/anthonyyoskovich/bezosbestfriends as the working directory.


In [2]:

import datapipeline
import importlib
importlib.reload(datapipeline)
from datapipeline import sentiment_frame, alldata, wholetext
import sentiment_helpers as sm
import warnings
import plotly.express as px
import nltk
from nltk.text import Text

warnings.simplefilter(action='ignore', category=FutureWarning)
stopwords = nltk.corpus.stopwords.words('english')

loading from cache
loading from cache


In [3]:
props = (
    sentiment_frame
    .loc[lambda x: x['year'] == 2010]
    .groupby("year")["color"]
    .value_counts(normalize=True)
    .reset_index()
    .sort_values(by="color")
)
ns = (
    sentiment_frame
    .groupby('year')['color']
    .value_counts()
    .reset_index(name='n')
    .sort_values(by="color")['n']
)
props['n'] = ns
fig = px.bar(
    y=[1] * props.shape[0],
    x="proportion",
    color="color",
    data_frame=props,
    # color_discrete_map=COLORMAP,
    orientation="h",
    height=250,
    template="none",
    labels=dict(proportion="", y=""),
    custom_data=['color']
)
fig.update_layout(
    barmode="stack",
    yaxis=dict(
        showticklabels=False,
        showgrid=False,
        zeroline=False,
        tickvals=[],
        ticktext=[],
    ),
    xaxis=dict(
        showticklabels=False,
        showgrid=False,
        zeroline=False,
        tickvals=[],
        ticktext=[],
    ),
)
fig.update_traces(
    texttemplate='%{x:.0%} %{customdata[0]}',
    textposition='inside',
    insidetextanchor='middle'
)
fig

In [4]:
import plotly.graph_objects as go
import plotly.express as px


from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import FreqDist
from nltk.text import Text
lemmer = WordNetLemmatizer()
lemmer.lemmatize('customer-focused')

'customer-focused'

In [11]:
alldata.iloc[0]['text_object'].concordance_list('customer')

[ConcordanceLine(left=['\nto', 'our', 'shareholder', 'amazon.com', 'passed', 'many', 'milestone', 'in', '1997', 'by', 'year', 'end', 'we', 'had', 'served', 'more', 'than', '15', 'million'], query='customer', right=['yielding', '838', 'revenue', 'growth', 'to', '1478', 'million', 'and', 'extended', 'our', 'market', 'leadership', 'despite', 'aggressive', 'competitive', 'entry', 'but', 'this'], offset=22, left_print='we had served more than 15 million', right_print='yielding 838 revenue growth to 147', line='we had served more than 15 million customer yielding 838 revenue growth to 147'),
 ConcordanceLine(left=['but', 'this', 'is', 'day', '1', 'for', 'the', 'internet', 'and', 'if', 'we', 'execute', 'well', 'for', 'amazon.com', 'today', 'online', 'commerce', 'save'], query='customer', right=['money', 'and', 'precious', 'time', 'tomorrow', 'through', 'personalization', 'online', 'commerce', 'will', 'accelerate', 'the', 'very', 'process', 'of', 'discovery', 'amazon.com', 'us'], offset=58, le

In [ ]:
alldata

In [34]:
_df[['index', 'year']]

,index,year
29610,29610,2008
35035,35035,2010
35050,35050,2010
37712,37712,2011
48400,48400,2014
54759,54759,2015
60340,60340,2016
60761,60761,2016
60983,60983,2016
61007,61007,2016


In [89]:
refline_data

,year,index
0,2007,26767
1,2010,35058
2,2011,37723
3,2012,40125
4,2013,44543
5,2014,48370
6,2015,52956
7,2016,58157
8,2017,64743
9,2018,69245


In [157]:
import plotly.io as pio
pio.templates.default = "simple_white"
SEARCH_TERM = 'aws'
base_data = (
    alldata
    # .assign(wordcount = lambda x: x['text_object'].apply(lambda x: x.count('customer')))
    .assign(split = lambda x: x['text_object'].apply(lambda x: x.tokens))
    .explode('split')
    .reset_index(drop=True)
    .reset_index()
)
summary_data = ( 
    base_data
    .loc[lambda x: x['split'] == SEARCH_TERM]
)
refline_data = base_data.groupby('year')['index'].min().reset_index()
fig = (
    summary_data
    .assign(static_value = 1)
    .assign(static_x = lambda x: x.apply(lambda x: [x['index'], x['year']], axis=1))
    .pipe(lambda x: px.scatter(
        data_frame=x, x='index', y='static_value',
    ))
)
for row in refline_data.to_dict(orient='records'):
    fig.add_vline(
        x=row['index'], 
        line_width=3, 
        line_dash="dash", 
        line_color="grey",
        annotation_text=row['year'],
        annotation_position='bottom right',
        annotation_textangle=45
    )

fig.update_traces(
    # use rbga so I can make the center opacity 
    # different
    marker = {
        'size': 20,
        'color': 'rgba(30,136,229,.25)',
        'line': {
            'color': 'rgba(0,0,0,1)',
            'width': 1
        }
    })
fig.update_layout(
    yaxis=dict(
        showticklabels=False,
        showgrid=False,
        zeroline=False,
        tickvals=[],
        ticktext=[],
    ),
    xaxis=dict(
        showticklabels=False,
        showgrid=False,
        zeroline=False,
        tickvals=[],
        ticktext=[],
    ),
    hovermode=False,
)
fig

In [2]:
import time
time._TimeTuple

AttributeError: module 'time' has no attribute '_TimeTuple'

In [162]:
_df = (
    alldata
    .assign(concordances = lambda x: x['text_object'].apply(lambda x: x.concordance_list(SEARCH_TERM)))
    .loc[lambda x: ~x['concordances'].isna()]
    [['pagenumber', 'year', 'author', 'concordances']]
    .explode('concordances')
    .loc[lambda x: ~x['concordances'].isna()]
    .assign(left_text = lambda x: x['concordances'].apply(lambda x: ' '.join(x.left)))
    .assign(right_text = lambda x: x['concordances'].apply(lambda x: ' '.join(x.right)))
    .assign(search_term = SEARCH_TERM)
    [['year', 'left_text', 'search_term', 'right_text']]
)
headers = _df.columns
fig = go.Figure(
    data=[
        go.Table(
            header=dict(
                # values=list(_df.columns),
                values=['Year', '', '', ''],
                fill_color="paleturquoise",
                align="left",
            ),
            cells=dict(
                values=[_df[col] for col in headers],
                fill_color="lavender",
                align=["left", 'right', 'center', 'left'],
            ),
            columnwidth=[50, 420, 40, 420],
        )
    ],
)
fig

In [6]:
# According to investing.com, amazon has somewhere around
# 300 million users: https://www.investing.com/academy/statistics/amazon-facts/
# Let's say Jeff wanted to get REAL personal with these letters, and, instead of using the word "customer",
# wanted to name one of these customers by their name:
# I have 2 questions: 
# 1. How many letters (following the current trend) could he produce before he ran out of unique names?
# 2. How long would it take to read each one?
from nltk.text import FreqDist
TOTAL_USERS = 300_000_000
customer_counts = (
    alldata
    .assign(n_cust = lambda x: x['text_object'].apply(lambda x: x.count('customer')))
    .groupby('year')
    ['n_cust'].sum()
)

hit_number = False
i = 1
total = 0
while total <= TOTAL_USERS:
    total += customer_counts.sum()
    i += 1
print('done')

done
